# Quantum benchmark

## I - Introduction

## II - Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import normalize
from sklearn.model_selection import cross_val_score, KFold
#Import classical libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.manifold import Isomap

plt.style.use('ggplot')

import functools

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
import logging

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer

from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
import qiskit

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## III - Data 

In [2]:
# Read out CSV

df = pd.read_csv('UCI_Credit_Card.csv', sep=',')

In [3]:
%%script false --no-raise-error

import sweetviz as sv

#EDA using Autoviz
sweet_report = sv.analyze(df)

#Saving results to HTML file
sweet_report.show_html('sweet_report.html')

## IV - Modelisation

### Classical

In [3]:
df_labels = df['default.payment.next.month']
df.drop(['default.payment.next.month'],axis = 1,inplace = True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df_labels, test_size=0.2, random_state=42)

---

## Quantum Approaches

In [5]:
std_scale = StandardScaler().fit(X_train)
sample_train = std_scale.transform(X_train)
sample_test = std_scale.transform(X_test)


In [6]:
iso = Isomap(n_components=2)

In [7]:
df_iso = iso.fit(sample_train, y_train)

In [8]:
df_iso = iso.transform(sample_train)

In [9]:
df_iso_train = pd.DataFrame(data = df_iso
             , columns = ['iso_1', 'iso_2'])


In [10]:
df_iso_train.head()

,iso_1,iso_2
0,-1.200149,-5.811421
1,3.949455,7.356078
2,-5.242927,0.317339
3,3.429907,6.341757
4,1.741190,-7.542940


In [11]:
n_dim = len(df_iso_train.columns)

## Split train test 

In [14]:
# Split dataset into train and test

sample_train, sample_test, label_train, label_test = train_test_split(
     df_iso_train, y_train, test_size=0.2, random_state=22)

sample_train = sample_train.to_numpy()
sample_test = sample_test.to_numpy()
# Normalize

#std_scale = StandardScaler().fit(sample_train)
#sample_train = std_scale.transform(sample_train)
#sample_test = std_scale.transform(sample_test)

# Scale for better fit within the feature map

samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Select a sample for a better control of the research and wall time

train_size = 800#160
sample_train = sample_train[:train_size]
label_train = y_train[:train_size]

test_size = 200 #40
sample_test = sample_test[:test_size]
label_test = y_test[:test_size]

In [15]:
sample_train[:2]

array([[-0.39243196, -0.16506998],
       [ 0.04804465,  0.27907479]])

In [16]:
sample_train.shape, label_train.shape

((800, 2), (800,))

In [17]:
# Basic parameters for hybrid model

seed = 8500
feature_dim = n_dim
num_reps = 2
num_shots =256 


## Hybrid

In [39]:
# Define feature_map

feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=num_reps)

# Define the backend
backend = QuantumInstance(
    BasicAer.get_backend("qasm_simulator"), shots=num_shots, seed_simulator=seed, seed_transpiler=seed
)

# Define the kernel

kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

# Model run
svc = SVC(kernel=kernel.evaluate)
#svc.fit(sample_train, label_train)
#score = svc.score(sample_test, label_test)

#print(f"Callable kernel classification test score: {score}")

In [ ]:
#result_predict = svc.predict(sample_test)

In [ ]:
#print(metrics.classification_report(label_test,result_predict))

## Classical Approaches

In [40]:
models = []
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('QSVC', svc))

In [41]:
df_results = pd.DataFrame()

In [42]:
_metrics = ['precision', 'recall', 'f1', 'accuracy',  'matthews_corrcoef','balanced_accuracy']

In [43]:
df_results = evaluate_ml_model(models, X_train, y_train, n_fold=10, metric=_metrics)

 83%|██████████████████████████████▊      | 5/6 [31:12:41<6:14:32, 22472.21s/it]


KeyboardInterrupt: 

In [ ]:
df_results

In [38]:
from sklearn.model_selection import cross_validate
from tqdm import tqdm
def evaluate_ml_model(_models, X, y, n_fold=10, metric='precision'):
    ''' Function to evaluate a ML and QML model with a list of metrics
    
    
    '''
    results = pd.DataFrame()
    kfold = KFold(n_splits=n_fold)
    columns = []
    for name, model in tqdm(_models):
        # -------------------
        # Variables initialization 
        _df = pd.DataFrame()
        names = []
        means = []
        stds = []
        
        # -------------------
        # k-fold Cross validation
        cv_results = cross_validate(model, X, y, cv=kfold, scoring=metric)
        
        # -------------------
        # Compute the mean and standard deviation 
        for _name, _array in cv_results.items():
            names.append(_name)
            means.append(round(100*_array.mean(), 2))
            stds.append(round(100*_array.std(), 2))
        # -------------------
        # Save the results in a dataframe 
        _df =  pd.DataFrame([means, stds], columns=names)
        columns.extend([name+' mean (%)', name+' std (%)'])
        #results = results.join(_df, on=_df.index)
        results = results.append(_df)
    results.index = columns
    print(results)
    return results

## Pennylane

In [20]:
from pennylane import numpy as np

In [21]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [22]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

#print(weights_init, bias_init)

In [23]:
sample_train

array([[-0.39243196, -0.16506998],
       [ 0.04804465,  0.27907479],
       [ 0.18141458, -0.43710395],
       ...,
       [-0.1980959 , -0.4089997 ],
       [-0.45984055, -0.177832  ],
       [-0.27884889,  0.27039532]])

In [24]:
circuit(weights_init, sample_train[0])

tensor(0.05181871, requires_grad=True)

In [25]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [26]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [27]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [28]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [29]:
Y = np.array(label_train * 2 - np.ones(len(label_train)),requires_grad=True)  # shift label from {0, 1} to {-1, 1}
X = np.array(sample_train, requires_grad=True)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.39243196, requires_grad=True), tensor(-0.16506998, requires_grad=True)], Y = -1
X = [tensor(0.04804465, requires_grad=True), tensor(0.27907479, requires_grad=True)], Y = -1
X = [tensor(0.18141458, requires_grad=True), tensor(-0.43710395, requires_grad=True)], Y = -1
X = [tensor(-0.43049507, requires_grad=True), tensor(-0.14768104, requires_grad=True)], Y = -1
X = [tensor(-0.15864416, requires_grad=True), tensor(0.39909446, requires_grad=True)], Y = -1


In [30]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0
ccost = 0 
for it in range(150):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    '''if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )'''
    prec = metrics.f1_score(Y, predictions, average='binary', pos_label=1)
    if  prec > abest or ((prec == abest) and (cost(weights, bias, X, Y) < ccost)):
        wbest = weights
        bbest = bias
        abest = prec
        ccost = cost(weights, bias, X, Y)
        print('New best')
    #prec = metrics.precision_score(Y, predictions, average='binary')
    print(
        "Iter: {:5d} | Cost: {:0.7f} | f1: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), prec
        )
    )
    
    

Iter:     1 | Cost: 0.7107961 | f1: 0.0000000 
Iter:     2 | Cost: 0.6886945 | f1: 0.0000000 
Iter:     3 | Cost: 0.8311803 | f1: 0.0000000 
Iter:     4 | Cost: 0.8051708 | f1: 0.0000000 
Iter:     5 | Cost: 0.7165882 | f1: 0.0000000 
Iter:     6 | Cost: 0.6965390 | f1: 0.0000000 
New best
Iter:     7 | Cost: 0.7986648 | f1: 0.1908127 
Iter:     8 | Cost: 0.7681767 | f1: 0.1105991 
Iter:     9 | Cost: 0.6864613 | f1: 0.0000000 
Iter:    10 | Cost: 0.7349488 | f1: 0.0000000 
Iter:    11 | Cost: 0.8214495 | f1: 0.0000000 
Iter:    12 | Cost: 0.8758459 | f1: 0.0000000 
Iter:    13 | Cost: 0.8893280 | f1: 0.0000000 
Iter:    14 | Cost: 0.8259435 | f1: 0.0000000 
Iter:    15 | Cost: 0.7247371 | f1: 0.0000000 
Iter:    16 | Cost: 0.6595238 | f1: 0.0000000 
Iter:    17 | Cost: 0.6917144 | f1: 0.0000000 
Iter:    18 | Cost: 0.8619374 | f1: 0.0000000 
New best
Iter:    19 | Cost: 0.9646267 | f1: 0.3144105 
Iter:    20 | Cost: 0.9179635 | f1: 0.0531915 
Iter:    21 | Cost: 0.8412988 | f1: 0.0000

In [33]:
Yte = np.array(label_test * 2 - np.ones(len(label_test)))
Xte = np.array(normalize(sample_test))

In [34]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.092605082863597, Accuracy: 44.0%


In [35]:
print(metrics.classification_report(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.39      0.78      0.52        76
         1.0       0.64      0.24      0.35       124

    accuracy                           0.45       200
   macro avg       0.51      0.51      0.43       200
weighted avg       0.54      0.45      0.41       200



In [36]:
print(metrics.classification_report(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.39      0.78      0.52        76
         1.0       0.64      0.24      0.35       124

    accuracy                           0.45       200
   macro avg       0.51      0.51      0.43       200
weighted avg       0.54      0.45      0.41       200



In [37]:
print(f'''

    Precision: {round(100*metrics.precision_score(predictions,Yte),2)}%
    Recall: {round(100*metrics.recall_score(predictions,Yte),2)}%
    f1: {round(100*metrics.f1_score(predictions,Yte),2)}%
    Accuracy: {round(100*metrics.accuracy_score(predictions,Yte),2)}%
    Balanced accuracy: {round(100*metrics.balanced_accuracy_score(predictions,Yte),2)}%
    Matthew corcorref: {round(100*metrics.matthews_corrcoef(predictions,Yte),2)}%
    ''')



    Precision: 63.83%
    Recall: 24.19%
    f1: 35.09%
    Accuracy: 44.5%
    Balanced accuracy: 50.91%
    Matthew corcorref: 2.09%
    
